In [27]:
import spacy
import random
import pickle
from spacy import displacy
import pandas as pd

# import spacy

In [28]:
model = spacy.load ( 'en_core_web_sm' )
model.pipe_names

['tagger', 'parser', 'ner']

# import skill set

In [29]:
df = pd.read_excel('Correlation_Link.xls')
skill_Set = set( list( df['node1'] ) + list( df['node2'] ) )
skill_Set = list(skill_Set)
skill_Set = skill_Set + ['JSP','J2EE','UNIX','SYBASE','Windows','LINUX','Computer Science','NoSQL','SQL Server']
skill_Set = [w.lower() for w in skill_Set]
skill_Set

['algorithms',
 'statistical software',
 'md',
 'matlab',
 'regression analysis',
 'support',
 'data analytics',
 'teradata sql',
 'mxnet',
 'sagemaker',
 's3',
 'elasticsearch',
 'bugs',
 'etl',
 'ruby',
 'software development',
 'c',
 'rds',
 'learning',
 'torch',
 'caffe',
 'aws',
 'data analytics',
 'data wrangling',
 'angular',
 'python scripting',
 'hr',
 'hive',
 'machine learning',
 'deep neural networks',
 'scripting',
 'deep learning',
 'automation',
 'microsoft access',
 'design',
 'hadoop',
 'analytics',
 'text analytics',
 'economics',
 'r studio',
 'software development',
 'database',
 'atlassian jira',
 'python r',
 'flask',
 'microsoft azure',
 'lsf',
 'python',
 'ms access',
 'confluence',
 'ibm db2',
 'mssql',
 'marketing',
 'cs',
 'java',
 'advanced analytics',
 'adobe analytics',
 'informatica',
 'sql server',
 'mathematics',
 'spotfire',
 'access',
 'sap hana',
 'predictive analytics',
 'cognitive computing',
 'microsoft word',
 'postgres',
 'microsoft visual studi

In [30]:
def is_tem_in_skill_set(ents, skill_Set):
    if str(ents).lower() in skill_Set:
        return True
    
    for tem in ents:
        if str(tem).lower() in skill_Set:
#             print( str(tem).lower() )
            return True
    
    return False

# import sentence contains DATE

In [31]:
with open('demo.pkl','rb') as out_data:
    data=pickle.load(out_data)

for sentence in data[:10]:
    print('-'*120)
    print(sentence)

------------------------------------------------------------------------------------------------------------------------
accurate status of projects aligned with the methodology being used (SCRUM/Iterative/ Waterfall)            Carry a Tier-2 support pager on a 6  8 week rotation           Support, maintenance, and development of JAVA web based and stand-alone applications
------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance
------------------------------------------------------------------------------------------------------------------------
- Minimum of 4-6 years working experience with Java/Unix/Oracle-based development.
-------------------

# test

In [42]:
sentence = data[ random.randint(0, len(data) ) ]

# sentence = 'Overall work experience:  1+ years with Java, J2EE application development.'
doc = model(sentence)
print( sentence )
print()

print( [chunks for chunks in doc.noun_chunks] )
print()

print( [chunks for chunks in doc.ents] )
print( [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)] )
print( [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text] )

Contract     TEKsystems has partnered with a local government integrator to support their need for a Java developer with 2-3 years of experience working in a J2EE environment.

[Contract     TEKsystems, a local government integrator, their need, a Java developer, 2-3 years, experience, a J2EE environment]

[Java, 2-3 years, J2EE]
[Java, J2EE]
[2-3 years]


# take a sentence plot dependency tree

In [43]:
def get_Date_attribute( string ):
    print( string )
    doc = model( string )

    print()
    print( [chunk for chunk in doc.noun_chunks] )
    displacy.render(doc, style='dep', jupyter=True)
    
#     print( [chunks for chunks in doc.noun_chunks if chunks.root.text in skill_Set] )
    print( [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)] )
    print( [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text] )

In [49]:
# import random

sentence = data[ random.randint(0, len(data) ) ]
get_Date_attribute( sentence )

Currently, they are 12 strong and looking to expand to 24-30 people by end of this year.   

[they, 24-30 people, end, this year]


[]
[]


In [46]:
# import random
# sentence = data[ random.randint(0, len(data) ) ]

sentence = 'SQL and RDBMS experience of at least two years.'
sentence = '3+ years scripting experience: BASH, Perl, Python, Ruby, etc.'
sentence = '(Minimum of 3 years exp), UNIX (3+ years), general SQL '
# sentence = '6 years experience programming Core Java, JSP and J2EE technologies.'
get_Date_attribute( sentence )

(Minimum of 3 years exp), UNIX (3+ years), general SQL 

[3 years, , UNIX, (3+ years, general SQL]


[UNIX, SQL]
[3+ years]


# Link year attribute to entity

In [56]:
def is_father( word, root ):
    if word == root:
        return True
    
    while word != word.head:
        word = word.head
        if word == root:
            return True
    
    return False

In [50]:
# sentence = '2+ years working in an Agile environment'
# sentence ='years experience with Java development'
# sentence = 'This candidate needs to have 3-5 years of Java development experience, J2EE experience, Spring, and Hibernate experience.'
# sentence = '2 years experience developing service-oriented (SOA) architectures and EAI technologies such as Tibco'
# sentence = 'Java application development experience  Must have 3+ years'

# sentence = 'Minimum 5 years experience in working within a web development team'
# sentence = data[ random.randint(0, len(data) ) ]

sentence = '3+ years experience of Java application development'

# testing

In [61]:
for sentence in data:
    doc = model( sentence )
    
#     print( [ent for ent in doc.noun_chunks] )
    date_list = [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text]
    tem_list = [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)]

    if len(date_list) > 0 and len(tem_list):
        print( '-' * 120 )
        print( sentence )
        print()
        
        print( 'date_list:', date_list )
        print( 'tem_list:', tem_list)

        for date in date_list:
            Date = date
            for tem in tem_list:
                Flag = False

                if is_father( tem.root, Date.root ):
                    print( tem ,'-->' , Date )
                    Flag = True

                if Flag == False:
                    Date_Head = Date.root
                    while Flag == False and Date_Head != Date_Head.head:
                        Date_Head = Date_Head.head

                        if Date_Head.tag_ == 'NOUN' and is_father( tem.root, Date_Head ):
                            print( tem ,'-->' , Date )
                            Flag = True

                        if Date_Head.tag_ == 'NOUN':
                            break



                    #####
                    if Date_Head.dep_ == 'compound':
                        Date_Head = Date_Head.head

                        if is_father( tem.root, Date_Head ):
                            print( tem ,'-->' , Date )
                            Flag = True

                        elif Date_Head.dep_ == 'compound':
                            Date_Head = Date_Head.head

                            if is_father( tem.root, Date_Head ):
                                print( tem ,'-->' , Date )
                                Flag = True

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
- Minimum of 4-6 years working experience with Java/Unix/Oracle-based development.

date_list: [4-6 years]
tem_list: [Java/Unix/Oracle]
------------------------------------------------------------------------------------------------------------------------
development experience with JSF, Servlets, JSP, JSTL, and IceFaces, and Velocity templates -3 years professional experience in developing w

------------------------------------------------------------------------------------------------------------------------
7+ years experience working in a Java/J2EE Development capacity.  

date_list: [7+ years]
tem_list: [Java/J2EE Development]
------------------------------------------------------------------------------------------------------------------------
The candidate must have a minimum 5-7 years of Java and J2EE  •Experience working with HTML/CSS/JavaScript  •Strong with JSP/Servlets/J2ee technology

date_list: [5-7 years]
tem_list: [Java, J2EE, HTML/CSS/JavaScript, JSP/Servlets/J2ee]
Java --> 5-7 years
J2EE --> 5-7 years
------------------------------------------------------------------------------------------------------------------------
Contract     TEKsystems has partnered with a local government integrator to support their need for a Java developer with 2-3 years of experience working in a J2EE environment.

date_list: [2-3 years]
tem_list: [Java, J2EE]
J2EE --> 2-3 ye

------------------------------------------------------------------------------------------------------------------------
Requirements: -4 year degree (Computer Science, Information Systems or relational functional field)

date_list: [year]
tem_list: [(Computer Science, Information Systems]
------------------------------------------------------------------------------------------------------------------------
- This position requires a Bachelor’s degree in Computer Science or Information Systems and at least 5+ years of experience in Web and Database Development, or a relevant combination of education and experience.

date_list: [at least 5+ years]
tem_list: [Computer Science or Information Systems, Database Development]
Database Development --> at least 5+ years
------------------------------------------------------------------------------------------------------------------------
They are looking for candidates with 0-2 years of Java Development experience.

date_list: [0-2 years]
tem

------------------------------------------------------------------------------------------------------------------------
 Minimum 5-7 years experience using Java/J2EE and OO design principles Experience with front-end technologies like JSP, HTML, CSS, JavaScript, Ajax Experience in web services like Rest, SOAP, JAX-RPC Experience working with relational databases, including Oracle, SQL Server, and MySQL Experience working with and implementing third party e-commerce services including, but not limited to: Credit card processing, Customer product reviews, Site analytics, SEO/SEM, and on-site searches Experience with Oracle E-Business Suite a plus  

date_list: [7 years]
tem_list: [Java/J2EE, JSP, JavaScript, Oracle, SQL, Oracle E-Business Suite]
------------------------------------------------------------------------------------------------------------------------
� Bachelor's Degree in Computer Science - minor in EE would be a plus  � 2+ years Java programming experience  � 2+ years ex

------------------------------------------------------------------------------------------------------------------------
Bachelors degree from a four year college or university in a related area 3 to 5 Years of java development experience Excellent object-oriented analysis, design and development skillsExperience with JSF/JSP, Servlets, Struts, JDBC, XML, SOAP, JavaScript and AjaxExperience in writing and optimizing complex SQL Knowledge of Apache, Tomcat strongly preferredKnowledge of Unix and/or Linuxknowledge of ANT, CVS, and EclipseAbility to accurately describe a solution design and to specify the required testingAbility to consistently deliver quality work that meets established coding standards and

date_list: [four year]
tem_list: [JSF/JSP, XML, JavaScript, SQL]
------------------------------------------------------------------------------------------------------------------------
At least 5 years in Java development           

date_list: [At least 5 years]
tem_list: [Java]
Ja

...

------------------------------------------------------------------------------------------------------------------------
At least 5 years experience with Java development Pluses:  

date_list: [At least 5 years]
tem_list: [Java]
------------------------------------------------------------------------------------------------------------------------
We are looking for a Lead JAVA Developer/Architect with 8+ years of experience in design and development of applications using Java script, Jquery and  J2EE technologies.     

date_list: [8+ years]
tem_list: [Java]
Java --> 8+ years
------------------------------------------------------------------------------------------------------------------------
Qualifications  • Bachelor´s degree in MIS or CS along with 2+ years of experience in Java / J2EE development   

date_list: [2+ years]
tem_list: [CS, Java / J2EE]
Java / J2EE --> 2+ years
----------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------
Java Developer Skills Required to be considered:  3+ years Java/J2EE Web development experience Significant practical experience of using spring/hibernate on projects Experienced in building

date_list: [3+ years]
tem_list: [Java Developer Skills, Java/J2EE Web]
------------------------------------------------------------------------------------------------------------------------
•              Minimum 5 years of comprehensive experience in web / enterprise application development in Microsoft Windows platform and related technologies •              In-depth experience with Java platform and related technologies •              

date_list: [5 years]
tem_list: [Microsoft, Windows, Java]
Microsoft --> 5 years
Windows --> 5 years
Java --> 5 years
------------------------------------------------------------------------------------------------------------------------
Jav

------------------------------------------------------------------------------------------------------------------------
The position also requires 3+ years experience in web application development and proficiency in writing servlets, JSPs, and EJBs, as well as a thorough knowledge of HTML, CSS, JavaScript, and AJAX.

date_list: [3+ years]
tem_list: [JavaScript]
------------------------------------------------------------------------------------------------------------------------
•              Minimum 5 years of comprehensive experience in web / enterprise application development in Microsoft Windows platform and related technologies •              In-depth experience with Java platform and related technologies •              

date_list: [5 years]
tem_list: [Microsoft, Windows, Java]
Microsoft --> 5 years
Windows --> 5 years
Java --> 5 years
------------------------------------------------------------------------------------------------------------------------
Java Developer Skills

------------------------------------------------------------------------------------------------------------------------
6+ years experience with UNIX and Windows operating systems.   

date_list: [6+ years]
tem_list: [UNIX, Windows]
------------------------------------------------------------------------------------------------------------------------
6+ years experience in JEE technologies including Servelts, JSP/JSF, JMS, JTA, JAXP/JAXB, JCA, JDBC, JPA, EJB and MDB.   

date_list: [6+ years]
tem_list: [JSP]
------------------------------------------------------------------------------------------------------------------------
3 or more years of Java experience designing and implementing Web based solutions in a J2EE Web environment.

date_list: [3 or more years]
tem_list: [Java]
Java --> 3 or more years
------------------------------------------------------------------------------------------------------------------------
3-4 years of hands on experience in ANSI SQL   2 -3 years of 

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
• Bachelor's Degree in Computer Science required, Masters a plus  • 3+ years of Java server development  • Knowledge/experience in developing Web Services using Service-Oriented concepts, standards and implementations  • Knowledge/experience in developing manageable, high-performance, and massively scalable application architectures distributed across multiple, heterogeneous platforms (e.g., h

In [197]:
def Extract_Info_from_JD( sentence ):
    skill_list = []
    skill_attribute_list = []
    
    doc = model( sentence )
    
    date_list = [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text]
    tem_list  = [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)]
    
    skill_list = tem_list
    if len(date_list) > 0 and len(tem_list):
        for date in date_list:
            Date = date
            for tem in tem_list:
                Flag = False

                if is_father( tem.root, Date.root ):
                    print( '****',tem, '-->', Date )
                    skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                    
                    Flag = True

                if Flag == False:
                    Date_Head = Date.root
                    while Flag == False and Date_Head != Date_Head.head:
                        Date_Head = Date_Head.head

                        if Date_Head.tag_ == 'NOUN' and is_father( tem.root, Date_Head ):
                            print( '****',tem, '-->', Date )
                            skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                            
                            Flag = True

                        if Date_Head.tag_ == 'NOUN':
                            break

                    #####
                    if Date_Head.dep_ == 'compound':
                        Date_Head = Date_Head.head

                        if is_father( tem.root, Date_Head ):
                            print( '****',tem, '-->', Date )
                            skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                            
                            Flag = True

                        elif Date_Head.dep_ == 'compound':
                            Date_Head = Date_Head.head

                            if is_father( tem.root, Date_Head ):
                                print( '****',tem, '-->', Date )
                                skill_attribute_list.append( str( str(tem) +'-->' + Date.text ) )
                                
                                Flag = True
                                
    return skill_list, skill_attribute_list

# import Job Description

In [198]:
import pandas as pd

nlp =  spacy.load('en')

data = pd.read_csv(r'E:/dataset/Top30.csv',usecols = ['Query','Description'])
data = data[data['Query'].isin([ 'Java Developer'])]
data.head(5)

,Query,Description
6,Java Developer,<P><STRONG>As a member of the Web and Portal D...
7,Java Developer,<BR>\r<TABLE border=0 cellSpacing=0 cellPaddin...
11,Java Developer,<strong>Application Developer-Senior-Java<br>\...
118,Java Developer,<b>Responsibilities:</b> Kforce is seeking a m...
132,Java Developer,<b>Responsibilities:</b> Our client is looking...


In [199]:
from bs4 import BeautifulSoup

def clean_text(text):
        text = text.replace(r'\\n', ' ').replace(r'\\r', ' ').replace(r'\\t', ' ').replace(r'\r', ' ').replace(r'\n', ' ').replace(r'\t', ' ').replace('\xa0','')

        soup = BeautifulSoup(text,'html.parser')
        return soup.get_text()

In [200]:
data['Description'] = data['Description'].apply(clean_text)
JD_set = list( data['Description'] )

# Test: get a random JD output

In [216]:
import random

text = JD_set[ random.randint(0, len(JD_set) ) ]
text = nlp( text )

total_skill_list = []
total_skill_attribute_list = []
for sent in text.sents:
    print( sent )
    skill_list ,skill_attribute_list = Extract_Info_from_JD( str(sent) )
    
    total_skill_list += skill_list
    total_skill_attribute_list += skill_attribute_list

print( '-' * 120 )
print( 'Extract Result:' )

print( total_skill_list )
for tem in total_skill_attribute_list:
    print( tem )

Adecco USA is a leader in recruiting and workforce solutions.
We have more than 900 offices in North America servicing a range of clients through an integrated suite of  workforce solutions.     
Combining niche industry expertise with the size and scale of Adecco SA, the world leader in workforce solutions, Adecco USA is made up of several specialty divisions that align with the unique needs of our clients.     
Java/J2EE Developer    
Adecco Technical has recently partnered with an industry leading fortune 500 company to bring on two Java/J2EE web application developers on a direct employment basis!  
This is not a contract or contract to hire scenario.  
The right candidate MUST be looking for a long term position with the opportunity for upward mobility.  
I am seeking true Object Oriented developers.  
No contractors by trade please...     
Must Have:     At least 3 years of experience developing Java/J2EE web applications  
**** Java/J2EE --> At least 3 years
JSP/HTML/JavaScript 